<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

Consider the problem at
https://en.wikipedia.org/wiki/Integer_programming#Example, which is
expressed as a maximization problem of two variables. Since `milp` requires
that the problem be expressed as a minimization problem, the objective
function coefficients on the decision variables are:


In [ ]:
import numpy as np
c = -np.array([0, 1])

Note the negative sign: we maximize the original objective function
by minimizing the negative of the objective function.

We collect the coefficients of the constraints into arrays like:


In [ ]:
A = np.array([[-1, 1], [3, 2], [2, 3]])
b_u = np.array([1, 12, 12])
b_l = np.full_like(b_u, -np.inf, dtype=float)

Because there is no lower limit on these constraints, we have defined a
variable ``b_l`` full of values representing negative infinity. This may
be unfamiliar to users of `scipy.optimize.linprog`, which only accepts
"less than" (or "upper bound") inequality constraints of the form
``A_ub @ x <= b_u``. By accepting both ``b_l`` and ``b_u`` of constraints
``b_l <= A_ub @ x <= b_u``, `milp` makes it easy to specify "greater than"
inequality constraints, "less than" inequality constraints, and equality
constraints concisely.

These arrays are collected into a single `LinearConstraint` object like:


In [ ]:
from scipy.optimize import LinearConstraint
constraints = LinearConstraint(A, b_l, b_u)

The non-negativity bounds on the decision variables are enforced by
default, so we do not need to provide an argument for `bounds`.

Finally, the problem states that both decision variables must be integers:


In [ ]:
integrality = np.ones_like(c)

We solve the problem like:


In [ ]:
from scipy.optimize import milp
res = milp(c=c, constraints=constraints, integrality=integrality)
res.x

[2.0, 2.0]

Note that had we solved the relaxed problem (without integrality
constraints):


In [ ]:
res = milp(c=c, constraints=constraints)  # OR:
# from scipy.optimize import linprog; res = linprog(c, A, b_u)
res.x

[1.8, 2.8]

we would not have obtained the correct solution by rounding to the nearest
integers.

Other examples are given :ref:`in the tutorial <tutorial-optimize_milp>`.